### Apple Silicon에서 TensorFlow-Metal 설치
* [Github](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/install/tensorflow-install-mac-metal-jul-2021.ipynb) 참고

In [1]:
# 시스템 확인
!brew install neofetch
!clear
!neofetch

To reinstall 7.1.0, run:
  brew reinstall neofetch
                    'c.
                 ,xNMM.
               .OMMMMo
               OMMM0,
     .;loddo:' loolloddol;.
   cKMMMMMMMMMMNWMMMMMMMMMM0:
 .KMMMMMMMMMMMMMMMMMMMMMMMWd.
 XMMMMMMMMMMMMMMMMMMMMMMMX.
;MMMMMMMMMMMMMMMMMMMMMMMM:
:MMMMMMMMMMMMMMMMMMMMMMMM:
.MMMMMMMMMMMMMMMMMMMMMMMMX.
 kMMMMMMMMMMMMMMMMMMMMMMMMWd.
 .XMMMMMMMMMMMMMMMMMMMMMMMMMMk
  .XMMMMMMMMMMMMMMMMMMMMMMMMK.
    kMMMMMMMMMMMMMMMMMMMMMMd
     ;KMMMMMMMWXXWMMMMMMMk.
       .cooc,.    .,coo:.
lee@jihos-MacBook-Pro.local 
--------------------------- 
OS: macOS 12.3.1 21E258 arm64 
Host: MacBookPro18,1 
Kernel: 21.4.0 
Uptime: 21 hours, 31 mins 
Packages: 138 (brew) 
Shell: zsh 5.8 
Resolution: 1728x1117, 2560x1440 
DE: Aqua 
WM: Quartz Compositor 
WM Theme: Blue (Dark) 
Terminal: vscode 
CPU: Apple M1 Pro 
GPU: Apple M1 Pro 
Memory: 3274MiB / 16384MiB 

                        
                        




In [2]:
# 버전 확인
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"TensorFlow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpus = tf.config.list_physical_devices('GPU')
print("GPU is", "available" if gpus else "NOT AVAILABLE")

Init Plugin
Init Graph Optimizer
Init Kernel
TensorFlow Version: 2.5.0
Keras Version: 2.5.0

Python 3.9.12 | packaged by conda-forge | (main, Mar 24 2022, 23:24:38) 
[Clang 12.0.1 ]
Pandas 1.4.2
Scikit-Learn 1.0.2
GPU is available


In [3]:
# 캐글 API 다운로드
!pip3 install kaggle

!mkdir -p ./data
!rm -rf ./data/*
!cd ./data && kaggle competitions download -c titanic && unzip titanic.zip

  0%|                                               | 0.00/34.1k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 34.1k/34.1k [00:00<00:00, 70.3MB/s]
Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [4]:
train_data = pd.read_csv("./data/train.csv")
train_data.head()

test_data = pd.read_csv("./data/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [16]:
train_features = train_data.copy()

test_features = test_data.copy()

def fill_nan(x):
    return x.fillna(x.mean())

train_features = fill_nan(train_features)
test_features = fill_nan(test_features)

features = [train_features, test_features]

for idx, feature in enumerate(features):
    for drop_column in ['PassengerId', 'Name', 'Cabin', 'Ticket']:
        feature.pop(drop_column)
    
    for to_one_hot_column in ['Sex', 'Embarked']:
        dummy = pd.get_dummies(feature[to_one_hot_column], prefix=to_one_hot_column)
                                
        # deep copy 방지
        if idx == 0:
            train_features = feature.join(dummy)
            feature = train_features
        else:
            test_features = feature.join(dummy)
            feature = test_features

        feature.pop(to_one_hot_column)

train_labels = train_features.pop('Survived')

train_features.head()
test_features.head()

/var/folders/22/28c1d33n2qq027g65rq1b89c0000gn/T/ipykernel_41334/3375295348.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return x.fillna(x.mean())


,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,34.5,0,0,7.8292,0,1,0,1,0
1,3,47.0,1,0,7.0000,1,0,0,0,1
2,2,62.0,0,0,9.6875,0,1,0,1,0
3,3,27.0,0,0,8.6625,0,1,0,0,1
4,3,22.0,1,1,12.2875,1,0,0,0,1


In [17]:
import numpy as np

train_features = np.asarray(train_features)
train_labels = np.asarray(train_labels)
test_features = np.asarray(test_features)

print(train_features)
print(train_labels)
print(test_features)

[[ 3.         22.          1.         ...  0.          0.
   1.        ]
 [ 1.         38.          1.         ...  1.          0.
   0.        ]
 [ 3.         26.          0.         ...  0.          0.
   1.        ]
 ...
 [ 3.         29.69911765  1.         ...  0.          0.
   1.        ]
 [ 1.         26.          0.         ...  1.          0.
   0.        ]
 [ 3.         32.          0.         ...  0.          1.
   0.        ]]
[0 1 1 1 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1
 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0
 1 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1
 0 1 1 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0
 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1
 1 0 1 0 0 0 0 0 1 1 1 0 1 1 0 

In [24]:
# 모델 정의
# https://www.tensorflow.org/guide/keras/sequential_model

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential(
    [
        layers.Dense(64),
        keras.layers.Dropout(.2),
        layers.Dense(16),
        keras.layers.Dropout(.2),
        layers.Dense(4),
        layers.Dense(1, activation='sigmoid')
    ]
)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(train_features, train_labels, epochs=150)

model.summary()

Epoch 1/150


2022-05-11 00:44:51.184282: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


28/28 [==============================] - 1s 10ms/step - loss: 3.0447 - accuracy: 0.5802
Epoch 2/150
28/28 [==============================] - 0s 12ms/step - loss: 2.2753 - accuracy: 0.6072
Epoch 3/150
28/28 [==============================] - 0s 7ms/step - loss: 1.7186 - accuracy: 0.6139
Epoch 4/150
28/28 [==============================] - 0s 6ms/step - loss: 1.2218 - accuracy: 0.6633
Epoch 5/150
28/28 [==============================] - 0s 7ms/step - loss: 1.2008 - accuracy: 0.6712
Epoch 6/150
28/28 [==============================] - 0s 7ms/step - loss: 0.8884 - accuracy: 0.6970
Epoch 7/150
28/28 [==============================] - 0s 7ms/step - loss: 0.8322 - accuracy: 0.7104
Epoch 8/150
28/28 [==============================] - 0s 7ms/step - loss: 0.7907 - accuracy: 0.7149
Epoch 9/150
28/28 [==============================] - 0s 6ms/step - loss: 0.6912 - accuracy: 0.7407
Epoch 10/150
28/28 [==============================] - 0s 6ms/step - loss: 0.6249 - accuracy: 0.7396
Epoch 11/150
28/28 

28/28 [==============================] - 0s 6ms/step - loss: 0.4899 - accuracy: 0.7946
Epoch 84/150
28/28 [==============================] - 0s 6ms/step - loss: 0.5066 - accuracy: 0.7856
Epoch 85/150
28/28 [==============================] - 0s 6ms/step - loss: 0.4952 - accuracy: 0.7800
Epoch 86/150
28/28 [==============================] - 0s 6ms/step - loss: 0.4722 - accuracy: 0.8036
Epoch 87/150
28/28 [==============================] - 0s 6ms/step - loss: 0.4674 - accuracy: 0.7946
Epoch 88/150
28/28 [==============================] - 0s 6ms/step - loss: 0.4706 - accuracy: 0.8002
Epoch 89/150
28/28 [==============================] - 0s 6ms/step - loss: 0.4651 - accuracy: 0.7969
Epoch 90/150
28/28 [==============================] - 0s 6ms/step - loss: 0.4900 - accuracy: 0.7890
Epoch 91/150
28/28 [==============================] - 0s 6ms/step - loss: 0.4626 - accuracy: 0.7924
Epoch 92/150
28/28 [==============================] - 0s 7ms/step - loss: 0.4796 - accuracy: 0.7868
Epoch 93/150


In [26]:
predicted_arr = model.predict(test_features)
predicted = test_data[['PassengerId']]
predicted['Survived'] = predicted_arr
predicted['Survived'] = predicted['Survived'].apply(lambda x: 1 if x >= 0.5 else 0)
predicted.head()

/var/folders/22/28c1d33n2qq027g65rq1b89c0000gn/T/ipykernel_41334/769173139.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicted['Survived'] = predicted_arr
/var/folders/22/28c1d33n2qq027g65rq1b89c0000gn/T/ipykernel_41334/769173139.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicted['Survived'] = predicted['Survived'].apply(lambda x: 1 if x >= 0.5 else 0)


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [27]:
predicted.to_csv('./data/submit.csv', index=False)

In [ ]:
dd